In [1]:
#import dependencies
import pandas as pd
import sqlite3

#display all columns in dfs
pd.set_option('display.max_columns', None)

In [2]:
#connect to SQLite db
conn = sqlite3.connect('../Data-and-DBs/pokedex.db')

In [3]:
#call all entries from the gens_1_to_6 table in the db
training_df = pd.read_sql('SELECT * FROM gens_1_to_6', conn)
training_df.head()

,NUMBER,CODE,SERIAL,NAME,TYPE1,TYPE2,COLOR,ABILITY1,ABILITY2,ABILITY_HIDDEN,GENERATION,LEGENDARY_FLAG,HEIGHT,WEIGHT,HP,ATK,DEF,SP_ATK,SP_DEF,SPD,TOTAL,CAPTURE_RATE
0,1,1,11,Bulbasaur,Grass,Poison,Green,Overgrow,None,Chrolophyll,1,0,0.7,6.9,45,49,49,65,65,45,318,45
1,2,1,21,Ivysaur,Grass,Poison,Green,Overgrow,None,Chrolophyll,1,0,1.0,13.0,60,62,63,80,80,60,405,45
2,3,1,31,Venusaur,Grass,Poison,Green,Overgrow,None,Chrolophyll,1,0,2.0,100.0,80,82,83,100,100,80,525,45
3,3,2,32,Mega Venusaur,Grass,Poison,Green,Thick Fat,None,None,1,0,2.4,155.5,80,100,123,122,120,80,625,45
4,4,1,41,Charmander,Fire,None,Red,Blaze,None,Solar Power,1,0,0.6,8.5,39,52,43,60,50,65,309,45


In [4]:
#pull necessary info from the gens_1_to_6 training df to create training data
X_train = training_df[['TYPE1', 'TYPE2', 'ABILITY1', 'ABILITY2', 'ABILITY_HIDDEN', 'HP', 'ATK', 'DEF', 
                 'SP_ATK', 'SP_DEF', 'SPD', 'TOTAL', 'CAPTURE_RATE']]
y_train = training_df['LEGENDARY_FLAG']
#double-check shapes of X and y match
print(X_train.shape, y_train.shape)

(817, 13) (817,)


In [5]:
#call all entries from the gen7 and gen_8 table in the db
test_df_1 = pd.read_sql('SELECT * FROM gen_7', conn)
test_df_2 = pd.read_sql('SELECT * FROM gen_8', conn)

In [6]:
#pull necessary info from the gen_7 and gen_8 testing dfs to create test data
X_test_1 = test_df_1[['TYPE1', 'TYPE2', 'ABILITY1', 'ABILITY2', 'ABILITY_HIDDEN', 'HP', 'ATK', 'DEF', 
                 'SP_ATK', 'SP_DEF', 'SPD', 'TOTAL', 'CAPTURE_RATE']]
y_test_1 = test_df_1['LEGENDARY_FLAG']

X_test_2 = test_df_2[['TYPE1', 'TYPE2', 'ABILITY1', 'ABILITY2', 'ABILITY_HIDDEN', 'HP', 'ATK', 'DEF', 
                 'SP_ATK', 'SP_DEF', 'SPD', 'TOTAL', 'CAPTURE_RATE']]
y_test_2 = test_df_2['LEGENDARY_FLAG']

#double-check shapes of X and y match
print(X_test_1.shape, y_test_1.shape)
print(X_test_2.shape, y_test_2.shape)

(118, 13) (118,)
(117, 13) (117,)


In [7]:
#import random forest test
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
#fit to the training data
rf = rf.fit(X_train, y_train)

ImportError: DLL load failed while importing _arpack: The specified procedure could not be found.